## Twitter Sentiment Analisys menggunakan metode A NAIVE BAYES CLASSIFIER (NBC)
### 1.Case Folding:
Case folding dilakukan untuk
mengubah setiap karakter didalam teks menjadi huruf
kecil. Tidak semua kata dalam teks konsisten dalam
penggunaan huruf kapital disinilah tujuan dilakukan case
folding untuk mengkonversi setiap karakter dalam kata
menjadi huruf kecil.
### 2.Tokenisasi: 
Tokenisasi merupakan proses
pemecahan kata pada suatu teks ke dalam satuan kata.
Tokenisasi dilakukan untuk menghasilkan kumpulan kata
yang berdiri sendiri, tokenisasi memecah teks yang
semula berupa kalimat menjadi kata-kata. tokenisasi
menghilangkan delimeter seperti titik (.), koma (,), spasi,
dan karakter angka yang ada pada kata tersebut.
Dalam penelitian ini tokenisasi dilakukan untuk
memecah kata, serta melakukan penghapusan delimeter
beserta karakter angka bersama tweet entity seperti
hashtag, retweet dan mention.
### 3.Filtering: 
Filtering merupakan proses dalam text
preprocessing setelah tokenisasi, filtering dilakukan untuk
untuk mengambil kata penting hasil tokenisasi. Pada tahap
filtering kata akan ditentukan apakah akan digunakan atau 
dibuang. Proses dalam filtering dalam membuang katakata yang tidak digunakan atau stopword terdapat dalam
bag of words stoplist.
Stopword merupakan daftar kata-kata yang tidak
mempresentasikan isi dari suatu dokumen teks, stopword
dilakukan untuk meghilangkan kata atau term yang tidak
memiliki arti. Daftar stoplist akan dibuat sebelum
melakukan proses stopword removal, jika kata-kata
terdapat dalam daftar stoplist, maka kata tersebut akan
dihapus, sehingga kata-kata yang tersisa akan dianggap
kata yang mencirikan isi suatu dokumen.
### 4.Stemming: 
Stemming merupakan proses mengubah
kata menjadi bentuk dasarnya. Stemming dilakukan untuk
meyeragamkan bentuk kata. Tujuan dari proses stemming
adalah menghilangkan imbuhan-imbuhan baik itu berupa
prefiks, sufiks, maupun konfiks yang ada pada setiap kata. 
Stemming dalam penelitian ini dilakukan berdasarkan
aturan morfologi bahasa Indonesia.


### berikut adalah langka-langkahnya:

### 1.Install Library(module) yang dibutuhkan

In [1]:
# %pip install tweepy
# %pip install Sastrawi
# %pip install numpy
# %pip install sklearn
# %pip install textblob
# %pip install pandas
# %pip install matplotlib

In [2]:
import tweepy

BEARERTOKEN = 'AAAAAAAAAAAAAAAAAAAAAJQrYQEAAAAA1BtlexYIrHRZvqkz6TEOraEC%2B0k%3D5cOp6U7ytEMpcTDctDQWVl6HFd0Y28pxhDnTGyHA9KiiC1MVCF'

client = tweepy.Client(bearer_token=BEARERTOKEN)

query = '"Elon Musk" lang:id'

# get data max 100
responses = client.search_recent_tweets(query=query, max_results=100, tweet_fields=['created_at'])
tweet100 = [tweet for tweet in responses.data]

# get max 1000
tweettexts = [tweet.text for tweet in tweepy.Paginator(client.search_recent_tweets, query=query, max_results=100).flatten(limit=1000)]

In [3]:
import pandas as pd
import numpy as np
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import nltk
import string
import re

In [4]:
data = {
    'text' : tweettexts
}
df = pd.DataFrame(data)
df.to_csv(f"crawling.csv", mode='w', index=False, header=data.keys())

In [5]:
import pandas as pd

df = pd.read_csv('crawling.csv', encoding='unicode_escape')
df.head(5)

,text
0,"RT @ayuazizz: Amber Heard punya gila, dia siap..."
1,"RT @ayuazizz: Amber Heard punya gila, dia siap..."
2,RT @banyuluynad: Cita2 pengen kayak warren buf...
3,tp rasanya taurus ni mmg ngam ngan cancer lah....
4,RT @swahilitimes: Jack Ma na Elon Musk wakijad...


In [6]:
#Cleaning Text
def textcleaner(text):
    new_text = str(text)
    
    # remove old style retweet text "RT"
    new_text = re.sub(r'^RT\s+', '', new_text)
    
    # remove username
    new_text = re.sub(r'@([A-Za-z0-9_]+)', '', new_text)

    # remove hyperlinks
    new_text = re.sub(r'https?:\/\/.*[\r\n]*', '', new_text)

    # remove invalid character
    new_text = re.sub("[^A-Za-z" "]+", " ", new_text)
    
    new_text    = re.sub(r'\b\w(1,2)\b',' ', new_text) #menghilangkan 2 kata
    
    return new_text

In [7]:
def caselower(sentences):
    return sentences.lower()

In [8]:
def tokenmaker(sentences):
    return sentences.strip().split()

In [9]:
#Stop Removal
stopword = nltk.corpus.stopwords.words("indonesian")
def remove_stopwords(text):
    text = [word for word in text if word not in stopword]
    return text

In [10]:
# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stemming(words):
    return [stemmer.stem(word) for word in words]

In [11]:
# create labeler

pos_kata = [pos.strip() for pos in open('kata_positif_2.txt', 'r').readlines()]
neg_kata = [neg.strip() for neg in open('kata_negatif_2.txt', 'r').readlines()]

#list kata-kata negasi
negasi = [neg.strip() for neg in open('negasi.txt', 'r').readlines()]


def labeling(stemmed):
    count_pos = 0
    count_neg = 0
    for ind, stem in enumerate(stemmed):
        if stem.strip() in pos_kata:
            if stemmed[ind-1].strip() in negasi:
                count_neg += 1
            else:
                count_pos += 1
        elif stem.strip() in neg_kata:
            if stemmed[ind-1].strip() in negasi:
                count_pos += 1
            else:
                count_neg += 1

    return 'NETRAL' if count_pos == count_neg else 'POSITIF' if count_pos > count_neg else 'NEGATIF'
    


In [12]:
# cleaning text
df['clean_text'] = df['text'].apply(lambda x : textcleaner(x))

In [13]:
# cover ke lower case
df['lower'] = df['clean_text'].apply(lambda x : caselower(x))

In [14]:
# tokenizing
df['token'] = df['lower'].apply(lambda x : tokenmaker(x))

In [15]:
# stopword removing
df["stop_removed"] = df['token'].apply(lambda x: remove_stopwords(x))

In [16]:
# stemming
df["stemmed"] = df['stop_removed'].apply(lambda x: stemming(x))
print("Stemming selesai!")

Stemming selesai!


In [17]:
# labelling sentiment
df["sentiment"] = df['stemmed'].apply(lambda x: labeling(x))
df.to_csv(f"labeled.csv", mode='w', index=False, header=False)
df.head(df.size)

,text,clean_text,lower,token,stop_removed,stemmed,sentiment
0,"RT @ayuazizz: Amber Heard punya gila, dia siap...",Amber Heard punya gila dia siap bekukan benih...,amber heard punya gila dia siap bekukan benih...,"[amber, heard, punya, gila, dia, siap, bekukan...","[amber, heard, gila, bekukan, benih, elon, mus...","[amber, heard, gila, beku, benih, elon, musk, ...",NEGATIF
1,"RT @ayuazizz: Amber Heard punya gila, dia siap...",Amber Heard punya gila dia siap bekukan benih...,amber heard punya gila dia siap bekukan benih...,"[amber, heard, punya, gila, dia, siap, bekukan...","[amber, heard, gila, bekukan, benih, elon, mus...","[amber, heard, gila, beku, benih, elon, musk, ...",NEGATIF
2,RT @banyuluynad: Cita2 pengen kayak warren buf...,Cita pengen kayak warren buffett jeff bezos e...,cita pengen kayak warren buffett jeff bezos e...,"[cita, pengen, kayak, warren, buffett, jeff, b...","[cita, pengen, kayak, warren, buffett, jeff, b...","[cita, ken, kayak, warren, buffett, jeff, bezo...",POSITIF
3,tp rasanya taurus ni mmg ngam ngan cancer lah....,tp rasanya taurus ni mmg ngam ngan cancer lah ...,tp rasanya taurus ni mmg ngam ngan cancer lah ...,"[tp, rasanya, taurus, ni, mmg, ngam, ngan, can...","[tp, taurus, ni, mmg, ngam, ngan, cancer, libr...","[tp, taurus, ni, mmg, ngam, ngan, cancer, libr...",NEGATIF
4,RT @swahilitimes: Jack Ma na Elon Musk wakijad...,Jack Ma na Elon Musk wakijadiliana kati ya bi...,jack ma na elon musk wakijadiliana kati ya bi...,"[jack, ma, na, elon, musk, wakijadiliana, kati...","[jack, ma, na, elon, musk, wakijadiliana, kati...","[jack, ma, na, elon, musk, wakijadiliana, kati...",NETRAL
...,...,...,...,...,...,...,...
995,"RT @ayuazizz: Amber Heard punya gila, dia siap...",Amber Heard punya gila dia siap bekukan benih...,amber heard punya gila dia siap bekukan benih...,"[amber, heard, punya, gila, dia, siap, bekukan...","[amber, heard, gila, bekukan, benih, elon, mus...","[amber, heard, gila, beku, benih, elon, musk, ...",NEGATIF
996,"RT @ayuazizz: Amber Heard punya gila, dia siap...",Amber Heard punya gila dia siap bekukan benih...,amber heard punya gila dia siap bekukan benih...,"[amber, heard, punya, gila, dia, siap, bekukan...","[amber, heard, gila, bekukan, benih, elon, mus...","[amber, heard, gila, beku, benih, elon, musk, ...",NEGATIF
997,"RT @ayuazizz: Amber Heard punya gila, dia siap...",Amber Heard punya gila dia siap bekukan benih...,amber heard punya gila dia siap bekukan benih...,"[amber, heard, punya, gila, dia, siap, bekukan...","[amber, heard, gila, bekukan, benih, elon, mus...","[amber, heard, gila, beku, benih, elon, musk, ...",NEGATIF
998,"RT @ayuazizz: Amber Heard punya gila, dia siap...",Amber Heard punya gila dia siap bekukan benih...,amber heard punya gila dia siap bekukan benih...,"[amber, heard, punya, gila, dia, siap, bekukan...","[amber, heard, gila, bekukan, benih, elon, mus...","[amber, heard, gila, beku, benih, elon, musk, ...",NEGATIF
